<a href="https://colab.research.google.com/github/tarunkpal/CNS/blob/main/phishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install levenshtein
!pip install python-whois
!pip install dnspython
!pip install tldextract
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
df=pd.read_csv("/content/dataset_B_05_2020.csv")
df.drop("url",axis=1,inplace=True)

In [ ]:
df.head()

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,37,19,0,3,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,77,23,1,1,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,126,50,1,4,1,0,1,2,0,3,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,18,11,0,2,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,55,15,0,2,2,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [ ]:
X=df.drop('status',axis=1).copy()
y=df['status'].copy()

In [ ]:
X.head()

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
0,37,19,0,3,0,0,0,0,0,0,...,0,0,1,0,45,-1,0,1,1,4
1,77,23,1,1,0,0,0,0,0,0,...,0,1,0,0,77,5767,0,0,1,2
2,126,50,1,4,1,0,1,2,0,3,...,0,1,0,0,14,4004,5828815,0,1,0
3,18,11,0,2,0,0,0,0,0,0,...,0,1,0,0,62,-1,107721,0,0,3
4,55,15,0,2,2,0,0,0,0,0,...,0,0,1,0,224,8175,8725,0,0,6


In [ ]:
X['ratio_intHyperlinks'].dtypes

dtype('float64')

In [ ]:
y.replace({'legitimate':0, 'phishing':1},inplace=True)

/tmp/ipython-input-12-3694566474.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y.replace({'legitimate':0, 'phishing':1},inplace=True)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    n_estimators=150,
    max_depth=5
)
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:12:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=150,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.968941382327209
Confusion Matrix:
 [[1104   39]
 [  32 1111]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97      1143
           1       0.97      0.97      0.97      1143

    accuracy                           0.97      2286
   macro avg       0.97      0.97      0.97      2286
weighted avg       0.97      0.97      0.97      2286



In [ ]:
from feature_extractor import extract_features

url = input("Enter a URL to extract features: ").strip()
print(f"Extracting features from: {url}")
features = extract_features(url)
print(features)
print(len(features))

if features:
    print(f"\nFeature vector for {url}:\n")
    for i, feature in enumerate(features[1:-1]):  # Skip URL and label
        print(f"Feature {i + 1}: {feature}")
    print(f"\nLabel: {features[-1]}")
else:
    print("Failed to extract features (URL may be inaccessible or malformed).")


Enter a URL to extract features: http://workbenefit.net
Enter the label (phishing/legitimate): legitimate
Extracting features from: http://workbenefit.net

Feature vector for http://workbenefit.net:

['http://workbenefit.net', 22, 15, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.0, 0.0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 1, 0, 1, 0, 11, 11, 0, 11, 11, 0, 11.0, 11.0, 0, 0, 0, 0, 0, 0, 0, 37, 0.972972972972973, 0.02702702702702703, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0, 100.0, 0, 100.0, 0.0, 0, 0, 0, 0.0, 0, 0, 0, 1, 0, 0, -1, 3, 0, 0, 1, 0, 'legitimate']
89


In [ ]:
fd = {}
column_names = [
     "length_url", "length_hostname", "ip", "nb_dots", "nb_hyphens",
    "nb_at", "nb_qm", "nb_and", "nb_or", "nb_eq", "nb_underscore",
    "nb_tilde", "nb_percent", "nb_slash", "nb_star", "nb_colon",
    "nb_comma", "nb_semicolumn", "nb_dollar", "nb_space", "nb_www",
    "nb_com", "nb_dslash", "http_in_path", "https_token", "ratio_digits_url",
    "ratio_digits_host", "punycode", "port", "tld_in_path", "tld_in_subdomain",
    "abnormal_subdomain", "nb_subdomains", "prefix_suffix", "random_domain",
    "shortening_service", "path_extension", "nb_redirection",
    "nb_external_redirection", "length_words_raw", "char_repeat",
    "shortest_words_raw", "shortest_word_host", "shortest_word_path",
    "longest_words_raw", "longest_word_host", "longest_word_path",
    "avg_words_raw", "avg_word_host", "avg_word_path", "phish_hints",
    "domain_in_brand", "brand_in_subdomain", "brand_in_path",
    "suspecious_tld", "statistical_report", "nb_hyperlinks",
    "ratio_intHyperlinks", "ratio_extHyperlinks", "ratio_nullHyperlinks",
    "nb_extCSS", "ratio_intRedirection", "ratio_extRedirection",
    "ratio_intErrors", "ratio_extErrors", "login_form", "external_favicon",
    "links_in_tags", "submit_email", "ratio_intMedia", "ratio_extMedia",
    "sfh", "iframe", "popup_window", "safe_anchor", "onmouseover",
    "right_clic", "empty_title", "domain_in_title", "domain_with_copyright",
    "whois_registered_domain", "domain_registration_length", "domain_age",
    "web_traffic", "dns_record", "google_index", "page_rank"
]
# features.pop()
print(len(features))
# data_row = [ 32, 24, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 2, 0, -1, 0, 0, 1, 0, 2, 0, 9, 9, 0, 10, 10, 0, 9.5, 9.5, 0, 0, 0, 0, 0, 0, 0, 31, 0.8064516129032258, 0.1935483870967742, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0, 1, 91.30434782608695, 0, 0.0, 100.0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, -1, 22, 0, 0, 1, 5]
data_row = features[1:]
print(len(data_row))
for i in range(0,87):
  fd[column_names[i]] = [data_row[i]]
fd


89
88


{'length_url': [22],
 'length_hostname': [15],
 'ip': [0],
 'nb_dots': [1],
 'nb_hyphens': [0],
 'nb_at': [0],
 'nb_qm': [0],
 'nb_and': [0],
 'nb_or': [0],
 'nb_eq': [0],
 'nb_underscore': [0],
 'nb_tilde': [0],
 'nb_percent': [0],
 'nb_slash': [2],
 'nb_star': [0],
 'nb_colon': [1],
 'nb_comma': [0],
 'nb_semicolumn': [0],
 'nb_dollar': [0],
 'nb_space': [0],
 'nb_www': [0],
 'nb_com': [0],
 'nb_dslash': [0],
 'http_in_path': [0],
 'https_token': [1],
 'ratio_digits_url': [0.0],
 'ratio_digits_host': [0.0],
 'punycode': [0],
 'port': [0],
 'tld_in_path': [0],
 'tld_in_subdomain': [0],
 'abnormal_subdomain': [0],
 'nb_subdomains': [1],
 'prefix_suffix': [0],
 'random_domain': [-1],
 'shortening_service': [0],
 'path_extension': [0],
 'nb_redirection': [1],
 'nb_external_redirection': [0],
 'length_words_raw': [1],
 'char_repeat': [0],
 'shortest_words_raw': [11],
 'shortest_word_host': [11],
 'shortest_word_path': [0],
 'longest_words_raw': [11],
 'longest_word_host': [11],
 'longest_

In [ ]:
f = pd.DataFrame(fd)
f

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
0,22,15,0,1,0,0,0,0,0,0,...,0,1,0,0,-1,3,0,0,1,0


In [ ]:
p = model.predict(f)
p

array([1])